In [1]:
import numpy as np
from scipy import stats
import math
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import pandas as pd
import geopandas as gpd

In [2]:
def custom_dist(u, v):
    if u[2] == v[2]:
        return sys.float_info.max
    else:
        return haversine([u[0], u[1]], [v[0], v[1]])

thresholds = {'CurbRamp': 0.0035,
                      'NoCurbRamp': 0.0035,
                      'SurfaceProblem': 0.01,
                      'Obstacle': 0.01,
                      'NoSidewalk': 0.01,
                      'Crosswalk': 0.01,
                      'Signal': 0.01,
                      'Occlusion': 0.01,
                      'Other': 0.01,
                      'Problem': 0.01}

def cluster(labels, label_type, thresholds):

    # Makes a normal dist matrix for a single user, but uses special dist function for multi-user clustering that
    # prevents the same user's attributes from being clustered together.
    dist_matrix = pdist(np.array(labels[['lat', 'lng', 'user_id']].values), custom_dist)
    link = linkage(dist_matrix, method='complete')
    curr_type = labels['label_type'].values

    # Copies the labels dataframe and adds a column to it for the cluster id each label is in.
    labelsCopy = labels.copy()
    labelsCopy.loc[:,'cluster_id'] = fcluster(link, t=thresholds[label_type], criterion='distance')

    # Cuts tree so that only labels less than clust_threth kilometers apart are clustered.
    clusters = labelsCopy.groupby('cluster_id')

    # Computes the center of each cluster and assigns temporariness and severity.
    cluster_list = [] # list of tuples (label_type, cluster_num, lat, lng, severity, temporary).
    for clust_num, clust in clusters:
        #ave_pos = np.mean(clust['coords'].tolist(), axis=0) # use ave pos of clusters.
        #ave_sev = None if pd.isnull(clust['severity']).all() else int(round(np.median(clust['severity'][~np.isnan(clust['severity'])])))
        #ave_temp = None if pd.isnull(clust['temporary']).all() else bool(round(np.mean(clust['temporary'])))

        cluster_list.append((curr_type, clust_num))

    cluster_df = pd.DataFrame(cluster_list, columns=['label_type', 'cluster_id'])

    return (cluster_df, labelsCopy)


# Pick which label types should be included in clustering, and which should be included in the "Problem" type.
label_types = ['CurbRamp', 'NoSidewalk', 'Occlusion', 'SurfaceProblem', 'Obstacle', 'Other', 'NoCurbRamp', 'Crosswalk', 'Signal']
# problem_types = ['SurfaceProblem', 'Obstacle']

# These are the columns required in the POST requests for the labels and clusters, respectively.
label_cols = ['label_id', 'label_type', 'cluster_id']
cluster_cols = ['label_type', 'cluster_id', 'lat', 'lng', 'severity', 'temporary']


def cluster_label_type_at_index(label_data,i):
        clusters_for_type_i = pd.DataFrame(columns=cluster_cols)
        labels_for_type_i = pd.DataFrame(columns=label_cols)

        label_type = label_types[i]
        # if label_type == 'Problem':
        #     type_data = label_data[label_data.label_type.isin(problem_types)]
        # else:
        type_data = label_data[label_data.label_type == label_type]

        # If there are >1 labels, we can do clustering. Otherwise just copy the 1 (or 0) labels.
        if type_data.shape[0] > 1:
            (clusters_for_type_i, labels_for_type_i) = cluster(type_data, label_type, thresholds)
        elif type_data.shape[0] == 1:
            labels_for_type_i = type_data.copy()
            labels_for_type_i.loc[:,'cluster_id'] = 1 # Gives the single cluster a cluster_id of 1.
            labels_for_type_i.loc[:,'label_type'] = label_type # Gives Problem type if needed.
            clusters_for_type_i = labels_for_type_i.filter(items=cluster_cols)

        return (label_type, clusters_for_type_i, labels_for_type_i)

In [3]:
df = gpd.read_file('../data/processed-labels/clusters/q3.shp')

In [4]:
df

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry
0,85414,NoSidewalk,5.0,Ml-KLIrMn1ZfBnbNK0NuIQ,1,197.125000,-6.687500,359.642944,-1.813416,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.564228,-122.312775,POINT (-122.31277 47.56423)
1,85424,NoCurbRamp,5.0,FOBAdLY7KV7QKCW6839LQQ,2,72.178574,-21.607143,29.547245,-1.009621,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.575085,-122.314636,POINT (-122.31464 47.57508)
2,85425,NoCurbRamp,5.0,FOBAdLY7KV7QKCW6839LQQ,1,297.178558,-32.000000,29.547245,-1.009621,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.575054,-122.314896,POINT (-122.31490 47.57505)
3,85427,NoCurbRamp,5.0,5zFwYo0tmsu9RkXbJsLZsQ,1,343.303558,-31.062500,349.068268,-0.112747,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.575218,-122.314796,POINT (-122.31480 47.57522)
4,85428,NoCurbRamp,5.0,5zFwYo0tmsu9RkXbJsLZsQ,1,48.178570,-35.000000,349.068268,-0.112747,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.575172,-122.314575,POINT (-122.31458 47.57517)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60376,231270,CurbRamp,1.0,R-sTl5O6sOdVPurgTWy3EQ,1,332.468750,-35.000000,215.130295,-0.157570,155af0a9-4149-4436-8aa3-7573e160036d,47.540771,-122.267258,POINT (-122.26726 47.54077)
60377,231285,NoSidewalk,5.0,N5MrAAtHZO2JGXUrYAYnSQ,3,0.890625,-11.656250,179.161057,-5.825714,155af0a9-4149-4436-8aa3-7573e160036d,47.552292,-122.263443,POINT (-122.26344 47.55229)
60378,231338,CurbRamp,1.0,d3oBrJ-nDtKMwlE7svc6pg,1,253.562500,-35.000000,177.959915,-0.962440,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563271,-122.281792,POINT (-122.28179 47.56327)
60379,231339,CurbRamp,1.0,AOIBJ12BjFanDJF1Ark05Q,1,110.312500,-35.000000,179.365723,-1.093400,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563469,-122.281654,POINT (-122.28165 47.56347)


In [5]:
df0 = cluster_label_type_at_index(df,0)[2]

In [6]:
df0

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry,cluster_id
5,85433,CurbRamp,1.0,2PjzoAUf1RHRm2XUcWqCag,1,134.053574,-34.625000,244.505478,2.585487,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.574997,-122.313232,POINT (-122.31323 47.57500),6008
7,85435,CurbRamp,1.0,tZMe3KCWZ9VeeIzBllC5Fg,1,50.991070,-20.562500,359.898193,-0.140648,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.575199,-122.313316,POINT (-122.31332 47.57520),3299
8,85439,CurbRamp,1.0,QDwsFAMYerNq-EbzCQVlLA,1,328.866058,-24.125000,181.814331,1.870056,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.576057,-122.313538,POINT (-122.31354 47.57606),1815
9,85569,CurbRamp,1.0,IXkOhk9ARdv1alGUJeZ6cA,2,35.772320,-33.125000,256.547119,-1.785950,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.584599,-122.310692,POINT (-122.31069 47.58460),3589
10,85579,CurbRamp,1.0,kWdtfyL797nc_QI9zTeJ5w,2,339.312500,-4.258929,359.871765,2.616417,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.586296,-122.309532,POINT (-122.30953 47.58630),7057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60368,231222,CurbRamp,1.0,gMZCijQWYviUCG588WlFTw,2,354.843750,-25.977678,52.083549,-0.176730,155af0a9-4149-4436-8aa3-7573e160036d,47.554150,-122.265961,POINT (-122.26596 47.55415),45
60376,231270,CurbRamp,1.0,R-sTl5O6sOdVPurgTWy3EQ,1,332.468750,-35.000000,215.130295,-0.157570,155af0a9-4149-4436-8aa3-7573e160036d,47.540771,-122.267258,POINT (-122.26726 47.54077),34
60378,231338,CurbRamp,1.0,d3oBrJ-nDtKMwlE7svc6pg,1,253.562500,-35.000000,177.959915,-0.962440,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563271,-122.281792,POINT (-122.28179 47.56327),31
60379,231339,CurbRamp,1.0,AOIBJ12BjFanDJF1Ark05Q,1,110.312500,-35.000000,179.365723,-1.093400,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563469,-122.281654,POINT (-122.28165 47.56347),24


In [7]:
# plot to check
# df0["clustered"] = df0.duplicated(subset="cluster_id", keep=False)
# df0.explore(tiles="cartodbpositron",column="clustered", categorical=True)

In [8]:
df1 = cluster_label_type_at_index(df,1)[2]

In [9]:
df2 = cluster_label_type_at_index(df,2)[2]

In [10]:
df3 = cluster_label_type_at_index(df,3)[2]

In [11]:
df4 = cluster_label_type_at_index(df,4)[2]

In [12]:
df5 = cluster_label_type_at_index(df,5)[2]

In [13]:
df6 = cluster_label_type_at_index(df,6)[2]

In [14]:
df7 = cluster_label_type_at_index(df,7)[2]

In [15]:
df8 = cluster_label_type_at_index(df,8)[2]

In [16]:
df0["clustered"] = df0.duplicated(subset="cluster_id", keep=False)
df1["clustered"] = df1.duplicated(subset="cluster_id", keep=False)
df2["clustered"] = df2.duplicated(subset="cluster_id", keep=False)
df3["clustered"] = df3.duplicated(subset="cluster_id", keep=False)
df4["clustered"] = df4.duplicated(subset="cluster_id", keep=False)
df5["clustered"] = df5.duplicated(subset="cluster_id", keep=False)
df6["clustered"] = df6.duplicated(subset="cluster_id", keep=False)
df7["clustered"] = df7.duplicated(subset="cluster_id", keep=False)
df8["clustered"] = df8.duplicated(subset="cluster_id", keep=False)

In [17]:
df0

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry,cluster_id,clustered
5,85433,CurbRamp,1.0,2PjzoAUf1RHRm2XUcWqCag,1,134.053574,-34.625000,244.505478,2.585487,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.574997,-122.313232,POINT (-122.31323 47.57500),6008,False
7,85435,CurbRamp,1.0,tZMe3KCWZ9VeeIzBllC5Fg,1,50.991070,-20.562500,359.898193,-0.140648,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.575199,-122.313316,POINT (-122.31332 47.57520),3299,True
8,85439,CurbRamp,1.0,QDwsFAMYerNq-EbzCQVlLA,1,328.866058,-24.125000,181.814331,1.870056,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.576057,-122.313538,POINT (-122.31354 47.57606),1815,False
9,85569,CurbRamp,1.0,IXkOhk9ARdv1alGUJeZ6cA,2,35.772320,-33.125000,256.547119,-1.785950,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.584599,-122.310692,POINT (-122.31069 47.58460),3589,False
10,85579,CurbRamp,1.0,kWdtfyL797nc_QI9zTeJ5w,2,339.312500,-4.258929,359.871765,2.616417,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.586296,-122.309532,POINT (-122.30953 47.58630),7057,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60368,231222,CurbRamp,1.0,gMZCijQWYviUCG588WlFTw,2,354.843750,-25.977678,52.083549,-0.176730,155af0a9-4149-4436-8aa3-7573e160036d,47.554150,-122.265961,POINT (-122.26596 47.55415),45,False
60376,231270,CurbRamp,1.0,R-sTl5O6sOdVPurgTWy3EQ,1,332.468750,-35.000000,215.130295,-0.157570,155af0a9-4149-4436-8aa3-7573e160036d,47.540771,-122.267258,POINT (-122.26726 47.54077),34,True
60378,231338,CurbRamp,1.0,d3oBrJ-nDtKMwlE7svc6pg,1,253.562500,-35.000000,177.959915,-0.962440,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563271,-122.281792,POINT (-122.28179 47.56327),31,False
60379,231339,CurbRamp,1.0,AOIBJ12BjFanDJF1Ark05Q,1,110.312500,-35.000000,179.365723,-1.093400,3ac3d4de-a70a-48ec-b46e-498d6e26ee6c,47.563469,-122.281654,POINT (-122.28165 47.56347),24,False


In [18]:
#concatenate all dataframes
clusters_q3 = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8])

In [19]:
clusters_q3

,label_id,label_type,severity,gsv_panora,zoom,heading,pitch,photograph,photogra_1,user_id,lat,lng,geometry,cluster_id,clustered
5,85433,CurbRamp,1.0,2PjzoAUf1RHRm2XUcWqCag,1,134.053574,-34.625000,244.505478,2.585487,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.574997,-122.313232,POINT (-122.31323 47.57500),6008,False
7,85435,CurbRamp,1.0,tZMe3KCWZ9VeeIzBllC5Fg,1,50.991070,-20.562500,359.898193,-0.140648,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.575199,-122.313316,POINT (-122.31332 47.57520),3299,True
8,85439,CurbRamp,1.0,QDwsFAMYerNq-EbzCQVlLA,1,328.866058,-24.125000,181.814331,1.870056,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.576057,-122.313538,POINT (-122.31354 47.57606),1815,False
9,85569,CurbRamp,1.0,IXkOhk9ARdv1alGUJeZ6cA,2,35.772320,-33.125000,256.547119,-1.785950,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.584599,-122.310692,POINT (-122.31069 47.58460),3589,False
10,85579,CurbRamp,1.0,kWdtfyL797nc_QI9zTeJ5w,2,339.312500,-4.258929,359.871765,2.616417,4f2a3a9a-a5d5-4d57-b2d1-6be61a6c2184,47.586296,-122.309532,POINT (-122.30953 47.58630),7057,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53202,204681,Signal,NaN,7JQDvPCBrge0eMyJn6paDw,3,231.991074,-8.060267,265.742798,-0.250572,ae9d3fab-6a0e-41c0-863e-ecff8ed43c7b,47.553368,-122.313622,POINT (-122.31362 47.55337),11,False
53255,206314,Signal,NaN,1Sw1pAR_FKEaDt-h71HxsQ,3,212.689728,-7.734375,278.561890,0.680862,ae9d3fab-6a0e-41c0-863e-ecff8ed43c7b,47.555126,-122.290222,POINT (-122.29022 47.55513),9,False
53858,209623,Signal,NaN,521qFBsGM8OM7_3vVrcClQ,2,160.825897,-8.142858,215.288620,1.195160,d57a2ee7-6b2c-4ef4-a04d-c8d688f78ca2,47.562687,-122.294823,POINT (-122.29482 47.56269),7,False
53863,209688,Signal,NaN,7XpJSENiFb70pWRG8A7plg,1,315.968750,-12.406250,175.600784,0.577698,d57a2ee7-6b2c-4ef4-a04d-c8d688f78ca2,47.564110,-122.282974,POINT (-122.28297 47.56411),5,False


In [20]:
#export to shapefile
clusters_q3.to_file("../data/processed-labels/clusters/clusters_q3.shp")